In [1]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='3'

In [3]:
import numpy as np
import pandas as pd 
import pickle
from helper_funcs import *
from umap import UMAP

from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
data_dir = '../Data/latent_vecs/'
files = [file for file in os.listdir(data_dir) if file.startswith('latent')]
files.sort()
files

['latent_vecs_CVAE_2021-09-15 14:03:53.826090',
 'latent_vecs_CVAE_2021-09-30 17:08:46.471890']

In [5]:

weights_folder = 'CVAE_2021-09-15 14:03:53.826090'

In [6]:
from importlib import reload
import make_models;reload(make_models);from make_models import *

batch_size = 32
cvae, z_encoder, s_encoder, cvae_decoder = get_fMRI_CVAE_4D(input_shape=(32,32,32,51),
                   latent_dim = 16,
                   batch_size = batch_size, 
                   disentangle = True, 
                   gamma = 1,
                   kernel_size = 3,
                   filters = 64,
                   intermediate_dim = 128,
                   nlayers = 2,
                   learning_rate=0.001)

In [10]:
os.path.join('../Assets/tf_weights/',weights_folder,'cvae_weights')

'../Assets/tf_weights/CVAE_2021-09-15 14:03:53.826090/cvae_weights'

In [9]:
cvae.load_weights(os.path.join('../Assets/tf_weights/',weights_folder,'cvae_weights'));


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv3D object at 0x1554941741c0> and <tensorflow.python.keras.layers.core.Flatten object at 0x155496cb7520>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv3D object at 0x15549410cd00> and <tensorflow.python.keras.layers.core.Flatten object at 0x1554941858e0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x155496ed6f70> and <tensorflow.python.keras.layers.core.Flatten object at 0x155494122e50>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x155496c91730> and <tensorflow.python.keras.layers.core.Dense object at 0x155496c91670>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x155496c91670> and <tensorflow.python.keras.layers.cor

ValueError: Shapes (16,) and (1,) are incompatible